<a href="https://colab.research.google.com/github/Juanba98/Algorithmic_Trading/blob/main/quantitative_momentum_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
import numpy as np
import pandas as pd
import math
import requests
from scipy.stats import percentileofscore as score 
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [ ]:
stocks = pd.read_csv('/content/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [ ]:
stocks['Ticker']

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, Length: 505, dtype: object

In [ ]:
from importlib.machinery import SourceFileLoader
from os.path import join
secrets = SourceFileLoader('secrets', '/content/secrets.py').load_module()


symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={secrets.IEX_CLOUD_API_TOKEN}'
response=requests.get(api_url).json()

response

{'companyName': 'Apple Inc',
 'marketcap': 2334538473111,
 'week52high': 190.48,
 'week52low': 130.05,
 'week52highSplitAdjustOnly': 188.5,
 'week52lowSplitAdjustOnly': 129.98,
 'week52change': 0.004435042934113802,
 'sharesOutstanding': 16488738429,
 'float': 0,
 'avg10Volume': 107796067,
 'avg30Volume': 102061680,
 'day200MovingAvg': 160.35,
 'day50MovingAvg': 164.98,
 'employees': 151194,
 'ttmEPS': 6.17,
 'ttmDividendRate': 0.9133810040240947,
 'dividendYield': 0.006415805558426367,
 'nextDividendDate': '',
 'exDividendDate': '2022-07-30',
 'nextEarningsDate': '2022-10-22',
 'peRatio': 23.53374193183436,
 'beta': 1.2882117098976995,
 'maxChangePercent': 54.64076959676893,
 'year5ChangePercent': 2.90415230642144,
 'year2ChangePercent': 0.24724517856146813,
 'year1ChangePercent': 0.013061885366841806,
 'ytdChangePercent': -0.200421840891921,
 'month6ChangePercent': -0.18215738682792487,
 'month3ChangePercent': 0.04494856519405921,
 'month1ChangePercent': -0.094218935690453,
 'day30Ch

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [ ]:
response['year1ChangePercent']

0.013061885366841806

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [ ]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [ ]:
dataFrame = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:

  batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={secrets.IEX_CLOUD_API_TOKEN}'

  data = requests.get(batch_api_call_url).json()

  for symbol in symbol_string.split(','):
   
    try:
      new_row = pd.Series(
                  [
                    symbol, 
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                  ],
                  index = my_columns

                ) 
      dataFrame = pd.concat(
        [
          dataFrame,
          new_row.to_frame().T
        ],
        ignore_index=True
      )
    except KeyError as e:
      print(f'Symbol {str(e)} data not found')

    

dataFrame
  

Symbol 'DISCA' data not found
Symbol 'HFC' data not found
Symbol 'VIAC' data not found
Symbol 'WLTW' data not found


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,126.9,-0.223014,N/A
1,AAL,12.51,-0.418605,N/A
2,AAP,161.47,-0.213872,N/A
3,AAPL,141.9,0.013078,N/A
4,ABBV,140.78,0.38496,N/A
...,...,...,...,...
496,YUM,108.15,-0.112292,N/A
497,ZBH,105.4,-0.267595,N/A
498,ZBRA,273.23,-0.487605,N/A
499,ZION,53.32,-0.148299,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [ ]:
dataFrame.sort_values('One-Year Price Return',ascending=False, inplace=True) #inplane to modify the dataFrame
dataFrame = dataFrame[:50]
dataFrame.reset_index(inplace=True)

dataFrame

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.93,2.337686,N/A
1,355,OXY,61.78,1.124596,N/A
2,147,DVN,62.32,0.84553,N/A
3,89,CF,98.1,0.804813,N/A
4,225,HRB,44.08,0.802213,N/A
5,298,MCK,346.52,0.756083,N/A
6,315,MRO,22.97,0.717816,N/A
7,42,APA,35.08,0.679431,N/A
8,313,MPC,104.03,0.661224,N/A
9,470,VRTX,298.11,0.631861,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [ ]:
#@markdown Enter the value of your portfolio.
portfolio_size = 10000000 #@param {type:"number"}

In [ ]:
position_size = float(portfolio_size)/len(dataFrame.index)
for i in range(0,len(dataFrame)):
  dataFrame.loc[i,'Number of Shares to Buy'] = math.floor(position_size/dataFrame.loc[i,'Price']) 

dataFrame

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.93,2.337686,2502
1,355,OXY,61.78,1.124596,3237
2,147,DVN,62.32,0.84553,3209
3,89,CF,98.1,0.804813,2038
4,225,HRB,44.08,0.802213,4537
5,298,MCK,346.52,0.756083,577
6,315,MRO,22.97,0.717816,8707
7,42,APA,35.08,0.679431,5701
8,313,MPC,104.03,0.661224,1922
9,470,VRTX,298.11,0.631861,670


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [ ]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataFrame = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:

  batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={secrets.IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()

  for symbol in symbol_string.split(','):
    try:
      if(not data[symbol]['stats']['month6ChangePercent']):
        print(f'Error: {symbol} has null stats')
        continue
      new_row = pd.Series(
                  [
                    symbol, 
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                  ], 
                  index = hqm_columns)
       
      hqm_dataFrame = pd.concat(
        [
          hqm_dataFrame,
          new_row.to_frame().T
        ],
        ignore_index=True
      )
    except KeyError as e:
      print(f'Symbol {str(e)} data not found')

hqm_dataFrame

Error: ALXN has null stats
Error: CERN has null stats
Error: CTL has null stats
Symbol 'DISCA' data not found
Error: DISCK has null stats
Error: ETFC has null stats
Error: FLIR has null stats
Symbol 'HFC' data not found
Error: MXIM has null stats
Error: MYL has null stats
Error: NBL has null stats
Error: PBCT has null stats
Symbol 'VIAC' data not found
Symbol 'WLTW' data not found
Error: XLNX has null stats


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,124.14,N/A,-0.225674,N/A,-0.072228,N/A,0.036401,N/A,-0.044498,N/A,N/A
1,AAL,12.51,N/A,-0.420029,N/A,-0.340185,N/A,-0.03513,N/A,-0.058558,N/A,N/A
2,AAP,157.46,N/A,-0.221734,N/A,-0.222642,N/A,-0.069804,N/A,-0.045803,N/A,N/A
3,AAPL,144.9,N/A,0.012986,N/A,-0.187858,N/A,0.044979,N/A,-0.097131,N/A,N/A
4,ABBV,137.17,N/A,0.38976,N/A,-0.107831,N/A,-0.061035,N/A,0.062817,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
485,YUM,106.46,N/A,-0.11013,N/A,-0.088388,N/A,-0.051189,N/A,-0.037552,N/A,N/A
486,ZBH,104.69,N/A,-0.265301,N/A,-0.162087,N/A,0.022218,N/A,0.009824,N/A,N/A
487,ZBRA,273.15,N/A,-0.491702,N/A,-0.358411,N/A,-0.099349,N/A,-0.122705,N/A,N/A
488,ZION,53.21,N/A,-0.144483,N/A,-0.206198,N/A,0.024927,N/A,-0.061787,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [ ]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataFrame.index:
  for time_period in time_periods:
    change_col = f'{time_period} Price Return'
    percentile_col = f'{time_period} Return Percentile'
    hqm_dataFrame.loc[row, percentile_col] = score(hqm_dataFrame[change_col], hqm_dataFrame.loc[row, change_col])/100




hqm_dataFrame


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,124.14,N/A,-0.225674,0.344898,-0.072228,0.755102,0.036401,0.761224,-0.044498,0.755102,N/A
1,AAL,12.51,N/A,-0.420029,0.093878,-0.340185,0.126531,-0.03513,0.538776,-0.058558,0.67551,N/A
2,AAP,157.46,N/A,-0.221734,0.34898,-0.222642,0.404082,-0.069804,0.385714,-0.045803,0.75102,N/A
3,AAPL,144.9,N/A,0.012986,0.732653,-0.187858,0.477551,0.044979,0.8,-0.097131,0.406122,N/A
4,ABBV,137.17,N/A,0.38976,0.946939,-0.107831,0.67551,-0.061035,0.438776,0.062817,0.977551,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
485,YUM,106.46,N/A,-0.11013,0.54898,-0.088388,0.716327,-0.051189,0.467347,-0.037552,0.797959,N/A
486,ZBH,104.69,N/A,-0.265301,0.277551,-0.162087,0.546939,0.022218,0.72449,0.009824,0.932653,N/A
487,ZBRA,273.15,N/A,-0.491702,0.042857,-0.358411,0.097959,-0.099349,0.273469,-0.122705,0.226531,N/A
488,ZION,53.21,N/A,-0.144483,0.495918,-0.206198,0.436735,0.024927,0.730612,-0.061787,0.661224,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [ ]:
from statistics import mean

for row in hqm_dataFrame.index:
  momentum_percentiles = []
  for time_period in time_periods:
    momentum_percentiles.append(hqm_dataFrame.loc[row, f'{time_period} Return Percentile'])
  hqm_dataFrame.loc[row, 'HQM Score'] =  mean(momentum_percentiles)

hqm_dataFrame

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,124.14,N/A,-0.225674,0.344898,-0.072228,0.755102,0.036401,0.761224,-0.044498,0.755102,0.654082
1,AAL,12.51,N/A,-0.420029,0.093878,-0.340185,0.126531,-0.03513,0.538776,-0.058558,0.67551,0.358673
2,AAP,157.46,N/A,-0.221734,0.34898,-0.222642,0.404082,-0.069804,0.385714,-0.045803,0.75102,0.472449
3,AAPL,144.9,N/A,0.012986,0.732653,-0.187858,0.477551,0.044979,0.8,-0.097131,0.406122,0.604082
4,ABBV,137.17,N/A,0.38976,0.946939,-0.107831,0.67551,-0.061035,0.438776,0.062817,0.977551,0.759694
...,...,...,...,...,...,...,...,...,...,...,...,...
485,YUM,106.46,N/A,-0.11013,0.54898,-0.088388,0.716327,-0.051189,0.467347,-0.037552,0.797959,0.632653
486,ZBH,104.69,N/A,-0.265301,0.277551,-0.162087,0.546939,0.022218,0.72449,0.009824,0.932653,0.620408
487,ZBRA,273.15,N/A,-0.491702,0.042857,-0.358411,0.097959,-0.099349,0.273469,-0.122705,0.226531,0.160204
488,ZION,53.21,N/A,-0.144483,0.495918,-0.206198,0.436735,0.024927,0.730612,-0.061787,0.661224,0.581122


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [ ]:
hqm_dataFrame.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataFrame = hqm_dataFrame[:50]
hqm_dataFrame.reset_index(drop=True, inplace=True)

hqm_dataFrame

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,83.59,N/A,2.283638,1.0,0.821994,1.0,0.223367,0.97551,0.081022,0.987755,0.990816
1,COG,22.72,N/A,0.357168,0.940816,0.183824,0.981633,0.262203,0.987755,0.259366,0.997959,0.977041
2,UNM,39.9,N/A,0.643514,0.981633,0.258596,0.991837,0.154451,0.95102,0.025149,0.959184,0.970918
3,HRB,44.64,N/A,0.823892,0.993878,0.698749,0.997959,0.247353,0.979592,-0.030344,0.842857,0.953571
4,MPC,102.26,N/A,0.668396,0.983673,0.175053,0.977551,0.2181,0.973469,-0.020576,0.863265,0.94949
5,ALB,272.03,N/A,0.223174,0.9,0.209032,0.985714,0.279075,0.991837,-0.00843,0.891837,0.942347
6,NLSN,28.07,N/A,0.482026,0.963265,0.023255,0.893878,0.207004,0.971429,-0.00368,0.910204,0.934694
7,FTI,8.62,N/A,0.121702,0.82449,0.091912,0.953061,0.261049,0.985714,0.030973,0.963265,0.931633
8,CAH,67.41,N/A,0.408516,0.95102,0.216606,0.987755,0.307307,0.993878,-0.042802,0.767347,0.925
9,VRTX,301.05,N/A,0.631152,0.979592,0.123763,0.967347,0.040946,0.783673,0.041128,0.967347,0.92449


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
#@markdown Enter the value of your portfolio.
portfolio_size = 10000000 #@param {type:"number"}

In [ ]:
position_size = float(portfolio_size)/len(hqm_dataFrame.index)

for row in hqm_dataFrame.index:
  hqm_dataFrame.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataFrame.loc[row, 'Price'])
hqm_dataFrame

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,83.59,2392,2.283638,1.0,0.821994,1.0,0.223367,0.97551,0.081022,0.987755,0.990816
1,COG,22.72,8802,0.357168,0.940816,0.183824,0.981633,0.262203,0.987755,0.259366,0.997959,0.977041
2,UNM,39.9,5012,0.643514,0.981633,0.258596,0.991837,0.154451,0.95102,0.025149,0.959184,0.970918
3,HRB,44.64,4480,0.823892,0.993878,0.698749,0.997959,0.247353,0.979592,-0.030344,0.842857,0.953571
4,MPC,102.26,1955,0.668396,0.983673,0.175053,0.977551,0.2181,0.973469,-0.020576,0.863265,0.94949
5,ALB,272.03,735,0.223174,0.9,0.209032,0.985714,0.279075,0.991837,-0.00843,0.891837,0.942347
6,NLSN,28.07,7125,0.482026,0.963265,0.023255,0.893878,0.207004,0.971429,-0.00368,0.910204,0.934694
7,FTI,8.62,23201,0.121702,0.82449,0.091912,0.953061,0.261049,0.985714,0.030973,0.963265,0.931633
8,CAH,67.41,2966,0.408516,0.95102,0.216606,0.987755,0.307307,0.993878,-0.042802,0.767347,0.925
9,VRTX,301.05,664,0.631152,0.979592,0.123763,0.967347,0.040946,0.783673,0.041128,0.967347,0.92449


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataFrame.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [ ]:
writer.save()

/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
